## Dependencies

In [32]:
# Tensorflow
import tensorflow as tf
print('Tested with TensorFLow 1.2.0')
print('Your TensorFlow version:', tf.__version__) 

# Feeding function for enqueue data
from tensorflow.python.estimator.inputs.queues import feeding_functions as ff

# Rnn common functions
from tensorflow.contrib.learn.python.learn.estimators import rnn_common

# Model builder
from tensorflow.python.estimator import model_fn as model_fn_lib

# Run an experiment
from tensorflow.contrib.learn.python.learn import learn_runner

# Helpers for data processing
import pandas as pd
import numpy as np
import argparse

Tested with TensorFLow 1.2.0
Your TensorFlow version: 1.2.0


## Parameters

## Helper functions

In [82]:
# data from: http://ai.stanford.edu/~amaas/data/sentiment/
TRAIN_INPUT = 'data/train.csv'
TEST_INPUT = 'data/test.csv'

# data manually generated
MY_TEST_INPUT = 'data/mytest.csv'

# Parameters for training
STEPS = 5000
BATCH_SIZE = 64

# Parameters for data processing
SEQUENCE_LENGTH_KEY = 'sequence_length'
REVIEW_KEY = 'review'
CLASSIFICATION_KEY = 'is_positive'

# Vocabulary size
VOCAB_FILE = 'data/vocab.txt'
VOCAB = [line[:len(line)-1] for line in open(VOCAB_FILE)]
VOCAB_SIZE = len(VOCAB) - 1 # embedding goes from [0, VOCAB_SIZE - 1]

print('there are %d words in the train and test files' % (VOCAB_SIZE + 1))

there are 62702 words in the train and test files


In [72]:
# This function creates a sparse tensor in the following way, given:
# indices = [[0, 0], [1, 1], [2, 2]]
# values = [1, 2, 3]
# dense_shape = [3, 4]
#
# The output will be a sparse tensor that represents this dense tensor:
# [ 
#   [1, 0, 0, 0]
#   [0, 2, 0, 0]
#   [0, 0, 3, 0]
# ]
#
# We're using this to generate a Sparse tensor that can be easily
# formated in a one hot representation.
# More at: https://www.tensorflow.org/api_docs/python/tf/SparseTensor
def _sparse_string_to_index(sp, mapping):
    # This operation constructs a lookup table to convert tensor of strings
    # into int64 IDs
    table = tf.contrib.lookup.index_table_from_tensor(mapping, dtype=tf.string)
    
    return tf.SparseTensor(indices=sp.indices,
                           values=table.lookup(sp.values),
                           dense_shape=sp.dense_shape)
def array_to_onehot(array, num_dim=2):
    array = np.asarray(array, dtype=np.int32)
    onehot = np.zeros([array.shape[0], num_dim])
    for i in range(array.shape[0]):
        onehot[i][array[i]] = 1
    return onehot

# Returns the column values from a CSV file as a list
def _get_csv_column(csv_file, column_name):
    with open(csv_file, 'r') as f:
        df = pd.read_csv(f)
        return df[column_name].tolist()

## Input functions

In [73]:
# Input function used for training and testing                                                
def get_input_fn(csv_file, batch_size, epochs=1):
    
    # loading csv in memory
    df = pd.read_csv(csv_file)
    df = df.dropna()
    
    # We can use a canned input function for generating batches
    # with the pandas data frame, then we call it and
    # make the changes needed on the features
    pd_fn = tf.estimator.inputs.pandas_input_fn(df, shuffle=True,
                                                batch_size=batch_size,
                                                num_epochs=epochs,
                                                queue_capacity=100000)

    def input_fn():   
        # calling the pandas input function
        fields = argparse.Namespace(**pd_fn())
        
        # Split sentences into words
        split_review = tf.string_split(fields.review, delimiter=' ')
        
        # Creating a tf constant to hold the word -> index
        # this is need to create the sparse tensor and after the one hot encode
        mapping = tf.constant(VOCAB, name="mapping")

        # Words represented in a sparse tensor
        integerized_review = _sparse_string_to_index(split_review, mapping)

        # Converting numbers to int 32
        classification = tf.cast(fields.is_positive, tf.int32)
        seq_len = tf.cast(fields.sequence_length, tf.int32)
        
        # reshaping outputs to correct shapes
        features = {REVIEW_KEY: integerized_review, SEQUENCE_LENGTH_KEY: seq_len}
        label_onehot = tf.one_hot(classification, 2)
        
        return features, label_onehot
    return input_fn

In [74]:
# Creating my own input function for a custom CSV file
# it's simpler than the input_fn above but just used for small tests
def my_test_input_fn():
    df = pd.read_csv(MY_TEST_INPUT)
    df = df.dropna()
    
    review = df.review.tolist()

    # Split sentences into words
    split_review = tf.string_split(review, delimiter=' ')
    # Creating a tf constant to hold the word -> index
    # this is need to create the sparse tensor and after the one hot encode
    mapping = tf.constant(VOCAB, name="mapping")

    # Words represented in a sparse tensor
    integerized_review = _sparse_string_to_index(split_review, mapping)

    x = {REVIEW_KEY: integerized_review, SEQUENCE_LENGTH_KEY: df.sequence_length.tolist()}
    y = df.is_positive.tolist()

    return x, y

In [75]:
train_input_fn = get_input_fn(TRAIN_INPUT, BATCH_SIZE, None)
test_input_fn = get_input_fn(TEST_INPUT, BATCH_SIZE)
my_test_input_fn = get_my_input_fn()

In [76]:
# Testing the input function
with tf.Graph().as_default():
    train_input = train_input_fn()
    with tf.train.MonitoredSession() as sess:
        #print (train_input)
        print (sess.run(train_input))
        pass

({'review': SparseTensorValue(indices=array([[ 0,  0],
       [ 0,  1],
       [ 0,  2],
       ..., 
       [63, 77],
       [63, 78],
       [63, 79]]), values=array([   37,   548,     0, ...,  8564,  2225, 10636]), dense_shape=array([64, 80])), 'sequence_length': array([80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80,
       80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80,
       80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 66, 80, 80, 80, 80, 80, 80,
       80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 53, 80, 80], dtype=int32)}, array([[ 1.,  0.],
       [ 1.,  0.],
       [ 1.,  0.],
       [ 1.,  0.],
       [ 1.,  0.],
       [ 0.,  1.],
       [ 1.,  0.],
       [ 1.,  0.],
       [ 0.,  1.],
       [ 0.,  1.],
       [ 0.,  1.],
       [ 0.,  1.],
       [ 1.,  0.],
       [ 0.,  1.],
       [ 0.,  1.],
       [ 1.,  0.],
       [ 1.,  0.],
       [ 1.,  0.],
       [ 0.,  1.],
       [ 0.,  1.],
       [ 1.,  0.],
       [ 1.,  0.],
       [ 1.,  0

## Creating the Estimator model

In [77]:
def get_model_fn(rnn_cell_sizes,
                 label_dimension,
                 dnn_layer_sizes=[],
                 optimizer='SGD',
                 learning_rate=0.01,
                 embed_dim=128):
    
    def model_fn(features, labels, mode):
        
        review = features[REVIEW_KEY]
        sequence_length = tf.cast(features[SEQUENCE_LENGTH_KEY], tf.int32)
        labels_onehot = labels
        
        # Creating dense representation for the sentences
        # and then converting it to embeding representation
        dense_review = tf.sparse_tensor_to_dense(review, default_value=VOCAB_SIZE)
        embed_review = tf.contrib.layers.embed_sequence(dense_review,
                                                        vocab_size=VOCAB_SIZE + 1,
                                                        embed_dim=embed_dim)
        
        
        # Each RNN layer will consist of a LSTM cell
        rnn_layers = [tf.contrib.rnn.LSTMCell(size) for size in rnn_cell_sizes]
        
        # Construct the layers
        multi_rnn_cell = tf.contrib.rnn.MultiRNNCell(rnn_layers)
        
        # Runs the RNN model dynamically
        # more about it at: 
        # https://www.tensorflow.org/api_docs/python/tf/nn/dynamic_rnn
        outputs, final_state = tf.nn.dynamic_rnn(cell=multi_rnn_cell,
                                                 inputs=embed_review,
                                                 sequence_length=sequence_length,
                                                 dtype=tf.float32)

        # Slice to keep only the last cell of the RNN
        last_activations = rnn_common.select_last_activations(outputs,
                                                              sequence_length)

        # Construct dense layers on top of the last cell of the RNN
        for units in dnn_layer_sizes:
            last_activations = tf.layers.dense(
              last_activations, units, activation=tf.nn.relu)
        
        # Final dense layer for prediction
        predictions = tf.layers.dense(last_activations, label_dimension)
        predictions_softmax = tf.nn.softmax(predictions)
        
        loss = None
        train_op = None
        
        preds_op = {
            'prediction': predictions_softmax,
            'label': labels_onehot
        }
        
        eval_op = {
            "accuracy": tf.metrics.accuracy(
                     tf.argmax(input=predictions_softmax, axis=1),
                     tf.argmax(input=labels_onehot, axis=1))
        }
        
        if mode != tf.contrib.learn.ModeKeys.INFER:    
            loss = tf.losses.softmax_cross_entropy(labels_onehot, predictions)
    
        if mode == tf.contrib.learn.ModeKeys.TRAIN:    
            train_op = tf.contrib.layers.optimize_loss(
              loss,
              tf.contrib.framework.get_global_step(),
              optimizer=optimizer,
              learning_rate=learning_rate)
        
        return tf.contrib.learn.ModelFnOps(mode,
                                           predictions=predictions_softmax,
                                           loss=loss,
                                           train_op=train_op,
                                           eval_metric_ops=eval_op)
    return model_fn

In [83]:
model_fn = get_model_fn(rnn_cell_sizes=[64], # size of the hidden layers
                        label_dimension=2, # since are just 2 classes
                        dnn_layer_sizes=[128, 64], # size of units in the dense layers on top of the RNN
                        optimizer='Adam',
                        learning_rate=0.001,
                        embed_dim=512)
estimator = tf.contrib.learn.Estimator(model_fn=model_fn, model_dir='tensorboard6/')

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fe46203abe0>, '_keep_checkpoint_max': 5, '_evaluation_master': '', '_environment': 'local', '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_save_checkpoints_secs': 600, '_is_chief': True, '_task_type': None, '_keep_checkpoint_every_n_hours': 10000, '_num_worker_replicas': 0, '_save_checkpoints_steps': None, '_num_ps_replicas': 0, '_task_id': 0, '_master': '', '_model_dir': 'tensorboard6/', '_session_config': None, '_save_summary_steps': 100, '_tf_random_seed': None}


## Create and Run Experiment

In [84]:
# create experiment
def generate_experiment_fn():
  
  """
  Create an experiment function given hyperparameters.
  Returns:
    A function (output_dir) -> Experiment where output_dir is a string
    representing the location of summaries, checkpoints, and exports.
    this function is used by learn_runner to create an Experiment which
    executes model code provided in the form of an Estimator and
    input functions.
    All listed arguments in the outer function are used to create an
    Estimator, and input functions (training, evaluation, serving).
    Unlisted args are passed through to Experiment.
  """

  def _experiment_fn(output_dir):
    return tf.contrib.learn.Experiment(
        estimator,
        train_input_fn=train_input_fn,
        eval_input_fn=test_input_fn,
        train_steps=STEPS
    )
  return _experiment_fn

In [85]:
# run experiment 
learn_runner.run(generate_experiment_fn(), '/tmp/outputdir')

Instructions for updating:
Monitors are deprecated. Please use tf.train.SessionRunHook.


/usr/local/lib/python3.5/site-packages/tensorflow/python/ops/gradients_impl.py:93: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into tensorboard6/model.ckpt.
INFO:tensorflow:loss = 0.693034, step = 1
INFO:tensorflow:Starting evaluation at 2017-06-21-15:26:02
INFO:tensorflow:Restoring parameters from tensorboard6/model.ckpt-1
INFO:tensorflow:Evaluation [1/100]
INFO:tensorflow:Evaluation [2/100]
INFO:tensorflow:Evaluation [3/100]
INFO:tensorflow:Evaluation [4/100]
INFO:tensorflow:Evaluation [5/100]
INFO:tensorflow:Evaluation [6/100]
INFO:tensorflow:Evaluation [7/100]
INFO:tensorflow:Evaluation [8/100]
INFO:tensorflow:Evaluation [9/100]
INFO:tensorflow:Evaluation [10/100]
INFO:tensorflow:Evaluation [11/100]
INFO:tensorflow:Evaluation [12/100]
INFO:tensorflow:Evaluation [13/100]
INFO:tensorflow:Evaluation [14/100]
INFO:tensorflow:Evaluation [15/100]
INFO:tensorflow:Evaluation [16/100]
INFO:tensorflow:Evaluation [17/100]
INFO:tensorflow:Evaluation [18/100]
INFO:tensorflow:Evaluation [19/100]
INFO:tensorflow:Evaluation [20/100]
INFO:

INFO:tensorflow:Evaluation [24/100]
INFO:tensorflow:Evaluation [25/100]
INFO:tensorflow:Evaluation [26/100]
INFO:tensorflow:Evaluation [27/100]
INFO:tensorflow:Evaluation [28/100]
INFO:tensorflow:Evaluation [29/100]
INFO:tensorflow:Evaluation [30/100]
INFO:tensorflow:Evaluation [31/100]
INFO:tensorflow:Evaluation [32/100]
INFO:tensorflow:Evaluation [33/100]
INFO:tensorflow:Evaluation [34/100]
INFO:tensorflow:Evaluation [35/100]
INFO:tensorflow:Evaluation [36/100]
INFO:tensorflow:Evaluation [37/100]
INFO:tensorflow:Evaluation [38/100]
INFO:tensorflow:Evaluation [39/100]
INFO:tensorflow:Evaluation [40/100]
INFO:tensorflow:Evaluation [41/100]
INFO:tensorflow:Evaluation [42/100]
INFO:tensorflow:Evaluation [43/100]
INFO:tensorflow:Evaluation [44/100]
INFO:tensorflow:Evaluation [45/100]
INFO:tensorflow:Evaluation [46/100]
INFO:tensorflow:Evaluation [47/100]
INFO:tensorflow:Evaluation [48/100]
INFO:tensorflow:Evaluation [49/100]
INFO:tensorflow:Evaluation [50/100]
INFO:tensorflow:Evaluation [

INFO:tensorflow:Evaluation [87/100]
INFO:tensorflow:Evaluation [88/100]
INFO:tensorflow:Evaluation [89/100]
INFO:tensorflow:Evaluation [90/100]
INFO:tensorflow:Evaluation [91/100]
INFO:tensorflow:Evaluation [92/100]
INFO:tensorflow:Evaluation [93/100]
INFO:tensorflow:Evaluation [94/100]
INFO:tensorflow:Evaluation [95/100]
INFO:tensorflow:Evaluation [96/100]
INFO:tensorflow:Evaluation [97/100]
INFO:tensorflow:Evaluation [98/100]
INFO:tensorflow:Evaluation [99/100]
INFO:tensorflow:Evaluation [100/100]
INFO:tensorflow:Finished evaluation at 2017-06-21-15:42:10
INFO:tensorflow:Saving dict for global step 5000: accuracy = 0.743594, global_step = 5000, loss = 1.54726


({'accuracy': 0.74359375, 'global_step': 5000, 'loss': 1.547258}, [])

## Making Predictions

In [81]:
preds = estimator.predict(input_fn=my_test_input_fn, as_iterable=True)

sentences = _get_csv_column(MY_TEST_INPUT, 'review')

print()
for p, s in zip(preds, sentences):
    print('sentence:', s)
    print('bad review:', p[0], 'good review:', p[1])
    print('-' * 10)

INFO:tensorflow:Restoring parameters from tensorboard5/model.ckpt-12284

sentence: this is a great movie
bad review: 0.00019553 good review: 0.999804
----------
sentence: this is a good movie but isnt the best
bad review: 0.0790645 good review: 0.920936
----------
sentence: this is a ok movie
bad review: 0.75468 good review: 0.24532
----------
sentence: this movie sucks
bad review: 0.999998 good review: 1.94564e-06
----------
sentence: this movie sucks but isnt the worst
bad review: 0.66062 good review: 0.33938
----------
sentence: its not that bad
bad review: 0.995739 good review: 0.00426115
----------
